# Virtual Fields Method

### Case Study: Unnotched Iosipescu Test for Orthotropic Material

**Based on:**
*“The Virtual Fields Method: Extracting Constitutive Mechanical Parameters from Full-Field Deformation Measurements”*
by F. Pierron and M. Grédiac

**Developed by:**
José Xavier
Universidade NOVA de Lisboa, NOVA FCT, UNIDEMI
[https://userweb.fct.unl.pt/~jmc.xavier/](https://userweb.fct.unl.pt/~jmc.xavier/)

## Course Context

This notebook is part of the **CISM Advanced School**
**“Image-Based Mechanics: An Overview of Experimental and Numerical Approaches”**
Udine, Italy — *6–10 October 2025*

**Coordinators:** Julien Réthoré and José Xavier

**Lecture Topic:**
*“The Virtual Fields Method: Extracting Material Parameters from Heterogeneous Fields — Hands-On Session”*

For more information:
[https://cism.it/en/activities/courses/C2516/](https://cism.it/en/activities/courses/C2516/)

---

## Introduction

In this exercise, you will implement the Virtual Fields Method (VFM) to identify orthotropic material properties from the Iosipescu test. The framework is provided, but you need to complete key sections.

**Learning Objectives:**
- Understand VFM principles for orthotropic materials
- Implement virtual field equations
- Build and solve the linear system for material identification
- Validate results against reference values

**What you need to complete:**
1. Virtual field strain expressions
2. System matrix assembly (A matrix and B vector)
3. Results validation

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
from scipy import io
from io import StringIO

# Set plotting parameters
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (12, 8)

## 1. VFM Theory for Orthotropic Materials

### Constitutive Relations

For an orthotropic material in plane stress:

$$
\begin{Bmatrix}
\sigma_{11} \\
\sigma_{22} \\
\sigma_{12}
\end{Bmatrix} =
\begin{bmatrix}
Q_{11} & Q_{12} & 0 \\
Q_{12} & Q_{22} & 0 \\
0 & 0 & Q_{66}
\end{bmatrix}
\begin{Bmatrix}
\varepsilon_{11} \\
\varepsilon_{22} \\
2\varepsilon_{12}
\end{Bmatrix}
$$

### VFM Equation

$$
Q_{11} \int_S \varepsilon_1\varepsilon_1^* \, dS + Q_{22} \int_S \varepsilon_2\varepsilon_2^* \, dS + Q_{12} \int_S (\varepsilon_1\varepsilon_2^* + \varepsilon_2\varepsilon_1^*) \, dS + Q_{66} \int_S \varepsilon_6\varepsilon_6^* \, dS = \int_{L_f} T_i u_i^* \, dl
$$

Where:
- $\varepsilon_i$ are the measured strains
- $\varepsilon_i^*$ are the virtual strains
- $u_i^*$ are the virtual displacements
- $T_i$ are the applied tractions

## 2. VFM Implementation Class

In [2]:
class IosipescuVFM:
    """
    VFM analysis with three virtual field sets for orthotropic materials
    """
    
    def __init__(self, data_source='csv'):
        self.data_source = data_source
        self.data = {}
        self.results = {}
    
    def load_data_from_csv(self, csv_dir='.'):
        """Load data from CSV files (local or URL)"""

        # Determine if we're working with URL or local path
        is_url = csv_dir.startswith('http')

        # Construct paths
        if is_url:
            base = csv_dir.rstrip('/')
            scalar_path = f"{base}/scalarsFE.csv"
            fem_path = f"{base}/FEM2VFM.csv"
        else:
            scalar_path = os.path.join(csv_dir, 'scalarsFE.csv')
            fem_path = os.path.join(csv_dir, 'FEM2VFM.csv')

        # Load scalar parameters
        try:
            if is_url:
                response = requests.get(scalar_path)
                response.raise_for_status()
                content = response.text
            else:
                with open(scalar_path, 'r') as f:
                    content = f.read()

            for line in content.split('\n'):
                if '=' in line:
                    key, value = line.strip().split('=')
                    key = key.strip()
                    value = float(value.strip())

                    if key == 'Length':
                        self.data['L'] = value
                    elif key == 'Width':
                        self.data['w'] = value
                    elif key == 'Thick':
                        self.data['t'] = value
                    elif key == 'P':
                        self.data['F'] = value

            print(f"Loaded scalars from: {scalar_path}")
        except Exception as e:
            print(f"Error loading {scalar_path}: {e}")
            raise

        # Load FEM data
        try:
            fem_data = pd.read_csv(fem_path, sep=r'\s+')

            self.data['X1'] = fem_data['X_Coord'].values
            self.data['X2'] = fem_data['Y_Coord'].values
            self.data['Eps1'] = fem_data['Eps_X'].values
            self.data['Eps2'] = fem_data['Eps_Y'].values
            self.data['Eps6'] = fem_data['Eps_XY'].values

            print(f"Loaded FEM data from: {fem_path}")
        except Exception as e:
            print(f"Error loading {fem_path}: {e}")
            raise

            self._validate_data()
    def _validate_data(self):
        """Validate loaded data"""
        print("Data validation:")
        for key, value in self.data.items():
            if np.isscalar(value):
                print(f"  {key}: {value}")
            else:
                arr = np.array(value)
                print(f"  {key}: shape {arr.shape}, range [{arr.min():.3f}, {arr.max():.3f}]")
    
    def virtual_fields_set_1(self, Eps1, Eps2, Eps6):
        """
        First set of virtual fields
        
        TODO: Complete this method by implementing the virtual field equations
        
        Virtual Fields:
        VF1: u1* = 0,  u2* = -x1
        VF2: u1* = x1(L-x1)x2,  u2* = x1³/3 - Lx1²/2
        VF3: u1* = 0,  u2* = x1(L-x1)x2
        VF4: u1* = L/(2π)sin(2πx1/L),  u2* = 0
        
        Steps:
        1. Extract geometric and force parameters
        2. Initialize A matrix (4x4) and B vector (4x1)
        3. For each virtual field, compute:
           - Virtual strains from displacement derivatives
           - Left side: integrals of ε*ε* (use np.mean for integration)
           - Right side: external virtual work
        4. Solve the linear system A*Q = B
        
        Returns:
        -------
        Q : numpy array
            [Q11, Q22, Q12, Q66] stiffness parameters
        """
        X1 = self.data['X1']
        X2 = self.data['X2']
        L = self.data['L']
        w = self.data['w']
        t = self.data['t']
        F = self.data['F']
        
        # TODO: Initialize A matrix and B vector
        A = np.zeros((4, 4))
        B = np.zeros(4)
        
        # ========================================
        # TODO: VF1 - Pure shear virtual field
        # ========================================
        # Virtual strains: ε1* = 0, ε2* = 0, ε6* = -1
        # Hint: A[0, 3] involves Q66 term
        # Hint: B[0] = external work on top surface
        
        # Your code here:
        # A[0, 3] = ...
        # B[0] = ...
        
        # ========================================
        # TODO: VF2 - Bending-like virtual field
        # ========================================
        # Virtual strains: ε1* = (L-2x1)x2, ε2* = 0, ε6* = 0
        # Hint: Use np.mean() for integration
        
        # Your code here:
        # vf2_eps1_star = ...
        # A[1, 0] = ...
        # A[1, 2] = ...
        # B[1] = ...
        
        # ========================================
        # TODO: VF3 - Mixed virtual field
        # ========================================
        # Virtual strains: ε1* = 0, ε2* = x1(L-x1), ε6* = (L-2x1)x2
        
        # Your code here:
        # vf3_eps2_star = ...
        # vf3_eps6_star = ...
        # A[2, 1] = ...
        # A[2, 2] = ...
        # A[2, 3] = ...
        # B[2] = ...
        
        # ========================================
        # TODO: VF4 - Sinusoidal virtual field
        # ========================================
        # Virtual strains: ε1* = cos(2πx1/L), ε2* = 0, ε6* = 0
        
        # Your code here:
        # vf4_eps1_star = ...
        # A[3, 0] = ...
        # A[3, 2] = ...
        # B[3] = ...
        
        # Solve the linear system
        Q = np.linalg.solve(A, B)
        return Q

## 3. Virtual Field Set 1 - Theory

### Virtual Displacement Fields

- **VF1:** $u_1^{*(1)} = 0, \quad u_2^{*(1)} = -x_1$
- **VF2:** $u_1^{*(2)} = x_1(L-x_1)x_2, \quad u_2^{*(2)} = \frac{x_1^3}{3} - \frac{Lx_1^2}{2}$
- **VF3:** $u_1^{*(3)} = 0, \quad u_2^{*(3)} = x_1(L-x_1)x_2$
- **VF4:** $u_1^{*(4)} = \frac{L}{2\pi} \sin(2\pi x_1/L), \quad u_2^{*(4)} = 0$

### Virtual Strain Fields (derive these from displacements!)

- **VF1:** $\varepsilon_1^{*(1)} = ?, \quad \varepsilon_2^{*(1)} = ?, \quad \varepsilon_6^{*(1)} = ?$
- **VF2:** $\varepsilon_1^{*(2)} = ?, \quad \varepsilon_2^{*(2)} = ?, \quad \varepsilon_6^{*(2)} = ?$
- **VF3:** $\varepsilon_1^{*(3)} = ?, \quad \varepsilon_2^{*(3)} = ?, \quad \varepsilon_6^{*(3)} = ?$
- **VF4:** $\varepsilon_1^{*(4)} = ?, \quad \varepsilon_2^{*(4)} = ?, \quad \varepsilon_6^{*(4)} = ?$

**Hint:** Remember that $\varepsilon_{11} = \frac{\partial u_1}{\partial x_1}$, $\varepsilon_{22} = \frac{\partial u_2}{\partial x_2}$, $\varepsilon_{12} = \frac{1}{2}(\frac{\partial u_1}{\partial x_2} + \frac{\partial u_2}{\partial x_1})$

## 4. Load and Process Data

In [3]:
# Initialize VFM analysis
vfm = IosipescuVFM(data_source='csv')

# Base URL
BASE_URL = 'https://userweb.fct.unl.pt/~jmc.xavier/CISM_C2516/1_VFM_Iosipescu_orthotropic_manuallyVFs/'

# Load data from current directory
vfm.load_data_from_csv('.')

try:
    vfm.load_data_from_csv(BASE_URL)
    print("Data loaded successfully from URL!")
except Exception as e:
    print(f"Error: {e}")

Loaded scalars from: ./scalarsFE.csv
Loaded FEM data from: ./FEM2VFM.csv
Loaded scalars from: https://userweb.fct.unl.pt/~jmc.xavier/CISM_C2516/1_VFM_Iosipescu_orthotropic_manuallyVFs/scalarsFE.csv
Loaded FEM data from: https://userweb.fct.unl.pt/~jmc.xavier/CISM_C2516/1_VFM_Iosipescu_orthotropic_manuallyVFs/FEM2VFM.csv
Data loaded successfully from URL!


## 5. Run Virtual Field Set 1

**TODO:** After completing the `virtual_fields_set_1` method, run this cell to compute the stiffness parameters.

In [ ]:
# Run VF Set 1
try:
    Q1 = vfm.virtual_fields_set_1(
        vfm.data['Eps1'],
        vfm.data['Eps2'],
        vfm.data['Eps6']
    )
    
    # Convert to GPa and display
    Q1_gpa = Q1 / 1e3
    
    print("\nVirtual Field Set 1 Results:")
    print("=" * 40)
    print(f"Q11 = {Q1_gpa[0]:.3f} GPa")
    print(f"Q22 = {Q1_gpa[1]:.3f} GPa")
    print(f"Q12 = {Q1_gpa[2]:.3f} GPa")
    print(f"Q66 = {Q1_gpa[3]:.3f} GPa")
except:
    print("\n⚠️  Complete the virtual_fields_set_1 method first!")
    Q1_gpa = None

## 6. Visualization of Strain Fields

In [ ]:
# Plot measured strain fields
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

X1 = vfm.data['X1']
X2 = vfm.data['X2']
strain_data = [vfm.data['Eps1'], vfm.data['Eps2'], vfm.data['Eps6']]
strain_labels = [r'$\varepsilon_1$', r'$\varepsilon_2$', r'$\varepsilon_6$']

for i, (data, label, ax) in enumerate(zip(strain_data, strain_labels, axes)):
    im = ax.tricontourf(X1, X2, data, levels=20, cmap='BrBG')
    ax.set_aspect('equal')
    ax.set_title(label, fontsize=14)
    ax.set_xlabel(r'$x_1$ (mm)')
    ax.set_ylabel(r'$x_2$ (mm)')
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## 7. Comparison with Reference Values

**TODO:** Compare your results with the reference values and compute the errors.

In [ ]:
if Q1_gpa is not None:
    # Reference values
    Q_ref = np.array([15.536, 1.965, 0.926, 1.109])  # GPa
    
    # TODO: Calculate percentage errors
    # errors = ...
    
    # Create comparison table
    results_df = pd.DataFrame({
        'Parameter': ['Q11', 'Q22', 'Q12', 'Q66'],
        'Computed (GPa)': Q1_gpa,
        'Reference (GPa)': Q_ref,
        # 'Error (%)': errors  # Uncomment after computing errors
    })
    
    print("\nComparison with Reference Values:")
    print(results_df.to_string(index=False))
    
    # TODO: Add a check if errors are within acceptable range (< 1%)
else:
    print("\n⚠️  Complete the implementation first to see results!")

## 8. Questions for Reflection

After completing the exercise, answer these questions:

1. **Physical Interpretation:** What does each virtual field represent physically? Why do we need four independent fields?

2. **Virtual Work:** Explain why only VF1 and VF2 have non-zero external virtual work.

3. **Numerical Integration:** Why is `np.mean()` appropriate for computing the integrals over the surface?

4. **Boundary Conditions:** How do the virtual fields satisfy kinematic admissibility?

5. **Error Analysis:** What are potential sources of error in your results?

6. **Advantages:** What are the main advantages of VFM compared to iterative optimization methods?

## Expected Results

If your implementation is correct, you should obtain:

- Q11 ≈ 15.54 GPa (error < 0.1%)
- Q22 ≈ 1.96 GPa (error < 0.3%)
- Q12 ≈ 0.92 GPa (error < 0.3%)
- Q66 ≈ 1.11 GPa (error < 0.01%)

## Tips for Success

1. **Start with VF1:** It's the simplest - pure shear case
2. **Check dimensions:** Make sure your units are consistent
3. **Derive strains carefully:** Double-check your derivatives
4. **Use print statements:** Debug by printing intermediate values
5. **Matrix symmetry:** Note that some matrix entries should be related